In [ ]:
import os
from cryptography import exceptions
from cryptography.hazmat import backends
from cryptography.hazmat.primitives.kdf import scrypt
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

backend = backends.default_backend()

# Crypto 101

## About Me

* Douglas Mendizábal
* Open Source Developer 
* I 💖 Python
* @douglas@mastodon.social
* @elrobotrojo on Twitter

# THIS IS NOT CRYPTO

![Cryptocurrency is not crypto](https://i.imgur.com/qMneYD0.jpg)




# Crypto == Cryptography
### (Don't @ Me)

# Cryptography (n.)

Practice and study of tecniques for secure communication in the presence of third parties called adversaries.

# This Presentation

* Talk about common use cases where you might need crypto

* Passwords

* Protecting Data at Rest

* Verifying data integrity

# Passwords

* Passwords Suck
* Until we figure out a better system we have to deal with them.

## How do we code a secure password system?

* We need to store passwords
* Storing them in plain text is bad, so we need crypto
* I heard somewhere you should *hash* them ... ?


# Cryptographic Hashing Functions

* One way function that maps arbitrary data to a fixed length bit string (hash or digest)

* MD5 - Broken

* SHA1 - Also Broken

* SHA-2 (SHA-256, SHA-512), SHA-3 - Still good.

* Can we use this to store passwords?

* Nope, Hashing is way too fast

In [ ]:
# This was supposed to be a demo of how to crack SHA-256 hashes,
# but I couldn't get the cracker in the next cell to work :(

import hashlib
import random

with open('/usr/share/dict/words') as f:
    words = f.read().splitlines()

passwords = list()
    
for i in range(500):
    pw = random.choice(words) + str(i)
    digest = hashlib.sha256(pw.encode('UTF-8')).hexdigest()
    passwords.append(digest)
    
print(passwords[0])

with open('passwords.txt', 'w') as f:
    f.write('\n'.join(words))

In [ ]:
for word in words:
    for pw in passwords:
        for x in range(500):
            guess = word + str(x)
            digest = hashlib.sha256(guess.encode('UTF-8')).hexdigest()
            if digest == pw:
                print("Found password {} for hash {}.".format(guess, pw))
                break

# Key Derivation Functions

* Derives one or more secrets (keys) from another secret
* More entropy from a low entropy key (e.g. Password ➜ Encryption Key)
* Multiple keys from one key

## Bcrypt

* Purposely designed to be slow
* Can be tuned to do more work - at least 12 rounds

In [ ]:
import bcrypt

password = b'Some secret pa$$w0rd'
hashed = bcrypt.hashpw(password, bcrypt.gensalt())

if bcrypt.checkpw(password, hashed):
    print('Password matches!')
else:
    print('Password does not match.')
    
print(hashed)

## Scrypt

* Like Bcrypt, it was designed to be slow
* Also has a tunable memory requirement

In [ ]:

salt = os.urandom(16)

kdf = scrypt.Scrypt(
    salt=salt,
    length=32,
    n=2**14,
    r=8,
    p=1,
    backend=backend
)

key = kdf.derive(b'Some secret pa$$w0rd')

print(key)

In [ ]:
kdf = scrypt.Scrypt(
    salt=salt,
    length=32,
    n=2**14,
    r=2**12,
    p=1,
    backend=backend
)

try:
    kdf.verify(b'Some secret pa$$w0r', key)
    print('Password matches!')
except exceptions.InvalidKey:
    print('Password does not match.')    

# About Randomness

* Always use `os.urandom`

# Protecting Data at Rest

# Hashing

* The best way to protect data is to not store it at all
* Useful if you just need to verify integrity, but don't need to store the actual data

# Symmetric Encryption

* Uses a single key for encryption and decryption
* Useful when you need to protect data that is not exposed to outside systems

In [ ]:
from cryptography import fernet

key = fernet.Fernet.generate_key()
print(key)

f = fernet.Fernet(key)

ciphertext = f.encrypt(b'Data to be encrypted')
print(ciphertext)

secret = f.decrypt(ciphertext)
print(secret)

# Asymmetric Encryption

* Uses multiple keys:  A public key you can share and a private key that must be kept secret
* Useful when you have to transfer data across trust boundaries

In [ ]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
    backend=backend
)

message = b'Data to be encrypted'

public_key = private_key.public_key()
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print(ciphertext)

In [ ]:
plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

if plaintext == message:
    print('Message data matches!')
else:
    print('Message data does not match.')

# Data Integrity

# Signing and Verification

* Signature algorithms can be built using other crypto primitives, usually a hash function
* They use asymmetric encription since signature usually need to be shared


In [ ]:
message = b'A message that needs to be signed'
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)
print(signature)

In [ ]:
try:
    public_key.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("Signature is valid!")
except exceptions.InvalidSignature:
    print("Signature does not validate.")


# Questions?